In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import load_model

def preprocess_image(image_path, img_size=(128, 128)):
    # Read image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")
    # Resize image
    img = cv2.resize(img, img_size)
    # Normalize image
    img = img / 255.0
    return img

def load_data(data_dir, img_size=(128, 128)):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                img = preprocess_image(img_path, img_size)
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Specify the path to your dataset directory
data_dir = '/content/drive/MyDrive/Signature_Dataset/dataset/sign_data/train'

X, y = load_data(data_dir)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

model.save('signature_recognition_model.h5')
model = load_model('signature_recognition_model.h5')

def predict_signature(image_path, true_label):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)  # Add the channel dimension
    prediction = model.predict(img)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0] == true_label

# Update the path to point to a specific image file and the true label of the signature
new_image_path = '/content/drive/MyDrive/Signature_Dataset/dataset/sign_data/test/049/01_049.png'
true_label = '001'  # The true label of the signature
is_genuine = predict_signature(new_image_path, true_label)
print(f'Signature is genuine: {is_genuine}')



Epoch 1/10
42/42 [==============================] - 36s 831ms/step - loss: 5.4976 - accuracy: 0.0083 - val_loss: 4.8519 - val_accuracy: 0.0091
Epoch 2/10
42/42 [==============================] - 38s 901ms/step - loss: 4.8499 - accuracy: 0.0114 - val_loss: 4.8523 - val_accuracy: 0.0152
Epoch 3/10
42/42 [==============================] - 33s 799ms/step - loss: 4.8481 - accuracy: 0.0099 - val_loss: 4.8526 - val_accuracy: 0.0091
Epoch 4/10
42/42 [==============================] - 34s 806ms/step - loss: 4.8465 - accuracy: 0.0159 - val_loss: 4.8530 - val_accuracy: 0.0091
Epoch 5/10
42/42 [==============================] - 35s 826ms/step - loss: 4.8448 - accuracy: 0.0159 - val_loss: 4.8532 - val_accuracy: 0.0091
Epoch 6/10
42/42 [==============================] - 35s 829ms/step - loss: 4.8433 - accuracy: 0.0121 - val_loss: 4.8537 - val_accuracy: 0.0091
Epoch 7/10
42/42 [==============================] - 35s 827ms/step - loss: 4.8418 - accuracy: 0.0159 - val_loss: 4.8539 - val_accuracy: 0.0091

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 78ms/step
Signature is genuine: False
